# Loading Polygon Data

Creating a connection the gridMET planetary dataset

In [1]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
asset

<Asset href=abfs://gridmet/gridmet.zarr>

Using xarray to read it:

In [2]:
import xarray as xr

ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)
ds

<xarray.Dataset>
Dimensions:                                    (time: 15341, lat: 585,
                                                lon: 1386, crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 49.4 ... 25.07
  * lon                                        (lon) float64 -124.8 ... -67.06
  * time                                       (time) datetime64[ns] 1979-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

Creating the Chicago polygon and time period:

In [3]:
polygon_dict = {
    'california': {
        'lat_min': 32.5,
        'lat_max': 42.0,
        'lon_min': -124.5,
        'lon_max': -114.1
    },
    'chicago': {
        'lat_min': 41.5781,
        'lat_max': 42.1781,
        'lon_min': -87.9298,
        'lon_max': -87.3298
    },
    'south_carolina': {
        'lat_min': 32.0,
        'lat_max': 35.2,
        'lon_min': -83.4,
        'lon_max': -78.5
    },
    'chicago_core': {
        'lat_min': 41.78,
        'lat_max': 41.93,
        'lon_min': -87.75,
        'lon_max': -87.55
    },
        'rochester': {
        'lat_min': 43.075,
        'lat_max': 43.275,
        'lon_min': -77.745,
        'lon_max': -77.505
    },
    'rochester_core': {
        'lat_min': 43.125,
        'lat_max': 43.185,
        'lon_min': -77.645,
        'lon_max': -77.545
    }
}

# Select which polygon to use
sel_polygon = 'rochester' 

# Extract bounding values from the dictionary
lat_min = polygon_dict[sel_polygon]['lat_min']
lat_max = polygon_dict[sel_polygon]['lat_max']
lon_min = polygon_dict[sel_polygon]['lon_min']
lon_max = polygon_dict[sel_polygon]['lon_max']


In [5]:
ds_polygon = ds.sel(
    time=slice("2015-01-01", "2019-12-31"),
    lat=slice(lat_max, lat_min),  # Note: latitudes are descending
    lon=slice(lon_min, lon_max)
).compute()

In [6]:
# Preview the selection
ds_polygon

<xarray.Dataset>
Dimensions:                                    (time: 1826, lat: 4, lon: 6,
                                                crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 43.23 ... 43.11
  * lon                                        (lon) float64 -77.72 ... -77.52
  * time                                       (time) datetime64[ns] 2015-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 273.4...
    burning_index_g                            (time, lat, lon) float32 52.0 ...
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 18.5 ...
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 15.5 ...
    mean_vapor_pressure_deficit                (time, lat, lon) float32 0.24 ...
    potential_evapotranspiration               (time, lat, lon) float32 1.6 ....
    precipitation_amount                       (time, lat, lon) float32 0.0 ....
    relative_humidity                          (time, lat, lon) float32 45.0 ...
    specific_humidity                          (time, lat, lon) float32 0.001...
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 64.7 ...
    wind_from_direction                        (time, lat, lon) float32 245.0...
    wind_speed                                 (time, lat, lon) float32 11.7 ...
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [7]:
# Estimate memory usage of the subset
total_bytes = sum(v.data.nbytes for v in ds_polygon.data_vars.values())
print(f"Estimated memory size for {sel_polygon} polygon_data: {total_bytes / 1e9:.2f} GB")


Estimated memory size for rochester polygon_data: 0.00 GB


## Weekly resampling each pair of (lat,long)

In [8]:
agg_methods = {
    "air_temperature": "mean",  # Average temperature per week
    "burning_index_g": "max",   # Use max for fire danger indicators
    "dead_fuel_moisture_1000hr": "mean",  # Slow-changing moisture → mean
    "dead_fuel_moisture_100hr": "mean",   # Same as above
    "mean_vapor_pressure_deficit": "mean",  # Continuous variable → mean
    "potential_evapotranspiration": "sum",  # Accumulated over time
    "precipitation_amount": "sum",  # Rainfall is cumulative
    "relative_humidity": "mean",  # Weekly average humidity
    "specific_humidity": "mean",  # Same idea
    "surface_downwelling_shortwave_flux_in_air": "mean",  # Radiative flux → mean
    "wind_from_direction": "mean",  # Can use circular stats, but mean works as approx
    "wind_speed": "mean"  # Weekly average wind speed
}

In [9]:
# Step 1: Extract only relevant variables
ds_subset = ds_polygon[list(agg_methods.keys())]

# Step 2: Create a new Dataset by applying appropriate method
ds_weekly = xr.Dataset({
    var: getattr(ds_subset[var].resample(time="1W"), method)()
    for var, method in agg_methods.items()
})

# Step 3 (optional): Inspect result
print(ds_weekly)

<xarray.Dataset>
Dimensions:                                    (lat: 4, lon: 6, time: 262)
Coordinates:
  * lat                                        (lat) float64 43.23 ... 43.11
  * lon                                        (lon) float64 -77.72 ... -77.52
  * time                                       (time) datetime64[ns] 2015-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 279.6...
    burning_index_g                            (time, lat, lon) float32 52.0 ...
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 19.2 ...
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 17.85...
    mean_vapor_pressure_deficit                (time, lat, lon) float32 0.307...
    potential_evapotranspiration               (time, lat, lon) float32 6.6 ....
    precipitation_amount                       (time, lat, lon) float32 17.2 ...
    relative_humidity                          (time, lat, lon) float32 42.

If we were to do this:
```python
df_weekly = ds_weekly.to_dataframe().reset_index()
``` 
gives: One row per week per lat-lon pair.

So if you have 14 latitudes × 14 longitudes = 196 grid points, you’ll get:

`210 weeks × 196 = 41,160 rows`

Each row corresponds to the climate values at a specific grid cell in Chicago for a specific week.

If we aggregate it for the **whole of Chicago:**

In [10]:
ds_weekly_avg = ds_weekly.mean(dim=["lat", "lon"])

## DataFraming it

Let's convert it into an actual DataFrame:

In [11]:
df_weekly_avg = ds_weekly_avg.to_dataframe().reset_index()

In [12]:
df_weekly_avg

,time,air_temperature,burning_index_g,dead_fuel_moisture_1000hr,dead_fuel_moisture_100hr,mean_vapor_pressure_deficit,potential_evapotranspiration,precipitation_amount,relative_humidity,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,wind_speed
0,2015-01-04,279.326050,46.875000,19.510416,18.075003,0.297500,6.095833,17.675001,43.272919,0.002618,45.330212,179.343750,7.440626
1,2015-01-11,267.082733,41.708332,20.551786,19.652977,0.101250,4.408333,7.533333,59.903576,0.001431,61.207745,254.035721,8.789882
2,2015-01-18,272.083344,31.583334,20.722025,19.851191,0.134048,4.775000,7.200000,51.393452,0.001811,81.877380,175.000000,5.262500
3,2015-01-25,271.151764,27.541666,20.469643,19.850595,0.074583,2.791667,0.000000,68.235115,0.002222,103.883331,251.803574,4.175000
4,2015-02-01,269.599396,27.541666,21.260714,21.279762,0.093274,3.945833,24.237501,55.754169,0.001616,97.035118,212.035721,5.333929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2019-12-08,267.105347,23.583334,23.203573,22.170832,0.097202,3.783334,15.712501,63.138691,0.002851,57.603577,220.482132,5.035120
258,2019-12-15,269.888702,26.083334,23.230951,20.536905,0.235655,7.824999,29.120834,49.863098,0.002957,55.451782,184.089294,6.445238
259,2019-12-22,265.060120,23.875000,23.398809,20.727976,0.150060,4.187500,8.958333,54.233932,0.002190,65.193451,198.232132,4.322619
260,2019-12-29,272.147034,26.166666,22.171427,18.663691,0.329940,8.724999,18.662500,44.522022,0.003273,76.335121,145.464294,4.561905


In [13]:
csv_path = f"data/{sel_polygon}_weekly_avg.csv"
parquet_path = f"data/{sel_polygon}_weekly_avg.parquet"

df_weekly_avg.to_csv(csv_path, index=False)
df_weekly_avg.to_parquet(parquet_path, index=False)

print(f"✅ Data saved to:\n- {csv_path}\n- {parquet_path}")

✅ Data saved to:
- data/rochester_weekly_avg.csv
- data/rochester_weekly_avg.parquet
